In [1]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import SGDClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline
from sklearn.metrics import confusion_matrix
import statsmodels.api as sm
import pandas as pd
import numpy as np
import re

from bag_of_words import *
from regex_predictor import *
from odds_ratio import *
from proximal import *

In [2]:
master_data = pd.read_csv('csv_files/master_data.csv')

### Bag of words linear regression to see if the text data carries any predictive information on heart failure.

In [10]:
text_clf = create_bow_model(max_words=10000, ngram_range=(1, 2))

text_clf.fit(master_data['notes'], master_data['heart_fail'])

/Users/jacobmchen/anaconda3/lib/python3.10/site-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


Pipeline(steps=[('vect',
                 CountVectorizer(max_features=10000, ngram_range=(1, 2))),
                ('tfidf', TfidfTransformer()),
                ('lr', LogisticRegression(C=1e+20))])

In [6]:
print('majority class', 1-np.mean(master_data['heart_fail']))

majority class 0.7359003089878102


In [11]:
predicted = text_clf.predict(master_data['notes'])

print(np.mean(predicted == master_data['heart_fail']))

0.8667277851346304


In [8]:
create_confusion_matrix(master_data['heart_fail'], predicted)

{'tn': 20193,
 'fp': 1480,
 'fn': 3012,
 'tp': 4766,
 'sensitivity': 0.6127539213165338,
 'specificity': 0.9317122687214506,
 'precision': 0.7630483509446045,
 'recall': 0.6127539213165338}

Find what words are used most often.

In [9]:
coef = text_clf['lr'].coef_[0]
print(coef)

words = text_clf['vect'].get_feature_names_out()
print(words)

sorted({w:c for w, c in zip(words, coef)}.items(), key=lambda kv: -kv[1])

[ 4.48809034  1.40679777 -0.30101529 ... -0.16443629 -2.14459195
  4.63673087]
['00' '00 m2' '00 test' ... 'yellow urine' 'yet' 'yo']


[('edema', 16.62324472999957),
 ('volume', 9.810191107285666),
 ('hypoxia', 9.345391061524824),
 ('12 23', 9.203377306821375),
 ('48 hr', 8.945215089652246),
 ('failure', 8.772028444071307),
 ('27 at', 8.431136782417367),
 ('congestive heart', 8.188796362455289),
 ('congestive', 8.146035888416783),
 ('rest left', 8.02876977685413),
 ('lvh moderately', 7.902885674241195),
 ('team', 7.830944078421591),
 ('heart failure', 7.753153682650851),
 ('on the', 7.732973320301035),
 ('reviewed of', 7.578037915038866),
 ('contractility the', 7.527921274998193),
 ('before', 7.51492918636692),
 ('basal inferior', 7.474279488961845),
 ('tx', 7.360186692522813),
 ('rv normal', 7.2230257572691565),
 ('aorta there', 7.178625168624511),
 ('lvef 20', 7.048062363244782),
 ('until', 7.02296360856155),
 ('depressed with', 6.988791341450554),
 ('iii', 6.9420815361667305),
 ('depressed lvef', 6.65835058950069),
 ('inferolateral wall', 6.5724674548516315),
 ('severe ar', 6.559499242504327),
 ('excluded the', 6.5

In [6]:
# atrial, ventricle, cardio, **valve
regex_predicted = regular_expression_predict(master_data['notes_half2'], ['valve'])

print(np.mean(regex_predicted))

print(np.mean(regex_predicted == master_data['heart_fail']))

create_confusion_matrix(master_data['heart_fail'], regex_predicted)

0.5566194696275168
0.5984856201826763


{'tn': 11453,
 'fp': 10220,
 'fn': 1605,
 'tp': 6173,
 'sensitivity': 0.7936487528927745,
 'specificity': 0.5284455313062336,
 'precision': 0.3765631672055145,
 'recall': 0.7936487528927745}